## Install apackages

In [18]:
!pip install -U langchain langgraph langchain-google-vertexai google-cloud-aiplatform vertexai

  Using cached vertexai-1.71.1-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of vertexai to determine which version is compatible with other requirements. This could take a while.
  Using cached vertexai-1.71.0-py3-none-any.whl.metadata (10 kB)
  Using cached vertexai-1.70.0-py3-none-any.whl.metadata (10 kB)
  Using cached vertexai-1.69.0-py3-none-any.whl.metadata (10 kB)
  Using cached vertexai-1.68.0-py3-none-any.whl.metadata (10 kB)
  Using cached vertexai-1.67.1-py3-none-any.whl.metadata (10 kB)
  Using cached vertexai-1.67.0-py3-none-any.whl.metadata (10 kB)
  Using cached vertexai-1.66.0-py3-none-any.whl.metadata (10 kB)
INFO: pip is still looking at multiple versions of vertexai to determine which version is compatible with other requirements. This could take a while.
  Using cached vertexai-1.65.0-py3-none-any.whl.metadata (10 kB)
  Using cached vertexai-1.64.0-py3-none-any.whl.metadata (10 kB)
  Using cached vertexai-1.63.0-py3-none-any.whl.metada

## Import libraries

In [19]:
from google.colab import auth
import google.cloud.aiplatform as aiplatform
import vertexai

from langchain_google_vertexai import VertexAI
from langgraph.graph import StateGraph
from typing import TypedDict

## Authenticate and setup project details

In [20]:
# Authenticate user
auth.authenticate_user()

# Setup project details
PROJECT_ID = '' # Enter Project details
REGION = 'us-central1'

aiplatform.init(project=PROJECT_ID, location=REGION)

# Initialize LLM
llm = VertexAI(model_name="gemini-pro")

## Define Agent State

In [21]:
class AgentState(TypedDict):
    input: str
    output: str

## Define Router function

In [22]:
# Router function returns the next node as a string
def router(state: AgentState) -> str:
    user_input = state["input"].lower()

    if "translate" in user_input:
        return "TranslatorAgent"
    elif "solve" in user_input or any(char.isdigit() for char in user_input):
        return "MathSolverAgent"
    else:
        return "SummarizerAgent"

## Define Agent Functions

In [23]:
# Summarizer agent function
def summarizer_agent(state: AgentState) -> AgentState:
    user_input = state["input"]
    prompt = f"You are an expert summarizer. Summarize this:\n\n{user_input}\n\nSummary:"
    output = llm.invoke(prompt)
    print("[SummarizerAgent] Output:", output)
    return {"input": user_input, "output": output}

In [24]:
# Translator agent function
def translator_agent(state: AgentState) -> AgentState:
    user_input = state["input"]
    prompt = f"You are a professional translator. Translate this to French:\n\n{user_input}\n\nFrench Translation:"
    output = llm.invoke(prompt)
    print("[TranslatorAgent] Output:", output)
    return {"input": user_input, "output": output}

In [25]:
# Math agent function
def math_solver_agent(state: AgentState) -> AgentState:
    user_input = state["input"]
    prompt = f"You are a math expert. Solve this problem and return only the numeric answer:\n\n{user_input}\n\nAnswer:"
    output = llm.invoke(prompt)
    print("[MathSolverAgent] Output:", output)
    return {"input": user_input, "output": output}

## Develop the Graph

In [26]:
# Create graph
graph = StateGraph(AgentState)

In [27]:
# Pass state from Router (entry point)
graph.add_node("Router", lambda state: state)

In [28]:
# Add agents
graph.add_node("SummarizerAgent", summarizer_agent)
graph.add_node("TranslatorAgent", translator_agent)
graph.add_node("MathSolverAgent", math_solver_agent)

In [29]:
# Conditional edges from Router to agents based on router decision
graph.add_conditional_edges("Router", router)

In [30]:
# After agents finish, end
graph.add_edge("SummarizerAgent", "__end__")
graph.add_edge("TranslatorAgent", "__end__")
graph.add_edge("MathSolverAgent", "__end__")

graph.set_entry_point("Router")

In [31]:
# Compile Graph
app = graph.compile()

## Invoke Agents

In [32]:
# Summarizer
response_summary = app.invoke({"input": "Artificial Intelligence is a field of computer science that focuses on creating intelligent machines."})
print("\n=== Summarizer Agent Response ===\n", response_summary["output"])

[SummarizerAgent] Output: ## Artificial Intelligence: A Summary

Artificial intelligence (AI) is a branch of computer science dedicated to simulating human intelligence in machines. This involves creating algorithms and systems that can reason, learn, solve problems, and understand language. 

The goal of AI is to develop machines that can perform tasks that typically require human intelligence, such as:

* **Problem-solving:** AI can be used to solve complex problems by analyzing data and identifying patterns. 
* **Decision-making:** AI can make decisions based on a set of rules or by learning from past experiences. 
* **Natural language processing:** AI can understand and generate human language, allowing for natural interaction between humans and machines. 
* **Machine learning:** AI can learn from data without explicitly being programmed, enabling it to adapt to new situations and improve its performance over time. 

AI has numerous applications in various fields, including healthc

In [33]:
# Translator
response_translation = app.invoke({"input": "Translate this sentence: Hello, how are you?"})
print("\n=== Translator Agent Response ===\n", response_translation["output"])

[TranslatorAgent] Output: Bonjour, comment allez-vous? 


=== Translator Agent Response ===
 Bonjour, comment allez-vous? 



In [34]:
# Math Solver
response_math = app.invoke({"input": "What is 1234 multiplied by 5678?"})
print("\n=== Math Solver Agent Response ===\n", response_math["output"])

[MathSolverAgent] Output: 7,043,732

=== Math Solver Agent Response ===
 7,043,732
